In [ ]:
from datetime import datetime, date, timedelta
from dateutil.parser import parse
import math
import sys
import time

import pandas as pd
import numpy as np
import geopandas as gpd

import matplotlib
import matplotlib.pyplot as plt
import mapclassify as mc

from pathlib import Path
import requests

In [ ]:
CM = 1/2.54
DEBUG = True

In [ ]:
today = datetime.today().strftime('%y%m%d')
covid_url = 'https://www.arcgis.com/sharing/rest/content/items/f10774f1c63e40168479a1feb6c7ca74/data'
covid_file = today + "_RKI_COVID19.csv"

In [ ]:
covid_path=Path(covid_file)
if not covid_path.is_file():
    print('Getting new Covid Data for ', today)
    r = requests.get(covid_url, allow_redirects=True)
    filesize = open(covid_path, 'wb').write(r.content)
    print('Done!')

In [ ]:
#Load the covid database and Einwohnerzahlen via pd.read_csv

#covid_path       = "211220_RKI_COVID19.csv"

landesdaten_path = "211217_RKI_Corona_Landkreise.csv"

covid = pd.read_csv(covid_file, parse_dates=['Meldedatum'])
ewz = pd.read_csv(landesdaten_path)

In [ ]:
# Geopandas datafiles, Bundesland und Kreise

bland_path = 'vg2500_geo84/vg2500_bld.shp'
bland = gpd.read_file(bland_path)

kreise_path = 'vg2500_geo84/vg2500_krs.shp' 
kreise = gpd.read_file(kreise_path)

#correct Göttingen

kreise.RS.replace({'03152': '03159'},inplace=True)
kreise.RS.replace({'03156': '03159'},inplace=True)

In [ ]:
#Einwohnerzahlen Bundesländer und Landkreise

bl=ewz.groupby('BL').EWZ_BL.max()
lk=ewz.groupby('county').EWZ.max()

summe_ew_deutschland = bl.sum()

if DEBUG:
    print('Einwohner Deutschland:', summe_ew_deutschland)

In [ ]:
#Tabelle der Einwohnerzahlen von Landkreisen, Bundesländern und Deutschland gesamt

einwohner=pd.concat([lk,bl])
einwohner.loc['Deutschland']=summe_ew_deutschland

In [ ]:
start=covid['Meldedatum'].min()
ende=covid['Meldedatum'].max()

landkreise=covid['Landkreis'].unique()
bundeslaender=covid['Bundesland'].unique()

if DEBUG:
    print('Start Erfassung:', start.strftime('%d.%m.%Y')) 
    print('Ende  Erfassung:', ende.strftime('%d.%m.%Y'))
    print('Anzahl Bundesländer:', len(bundeslaender))
    print('Anzahl Landkreise:', len(landkreise))

In [ ]:
#covid.Altersgruppe.unique()

In [ ]:
#Filter by age
#covid=covid[covid.Altersgruppe=='A15-A34']

In [ ]:
# Main grouping steps for Landkreise, Bundesländer und Deutschland gesamt

Faelle_lk = covid.groupby(['Landkreis', 'Meldedatum']).AnzahlFall.sum()

Faelle_bl = covid.groupby(['Bundesland', 'Meldedatum']).AnzahlFall.sum()

Faelle_d = covid.groupby('Meldedatum').AnzahlFall.sum()

In [ ]:
tagesliste = pd.date_range(start,ende)
result=pd.DataFrame(index=tagesliste)

lk_auswahl=landkreise
for i in lk_auswahl:
    tmp=Faelle_lk[i]
    result = pd.concat([result, tmp.rename(i)], axis=1)

bl_auswahl=bundeslaender
for i in bl_auswahl:
    tmp=Faelle_bl[i]
    result = pd.concat([result, tmp.rename(i)], axis=1)
    
result = pd.concat([result,Faelle_d.rename('Deutschland')], axis=1)



#Rollierende 7 Tage Mittelwerte und Summationen    
    
result.fillna(value=0, inplace=True)
result_m7 = result.rolling(7).mean()
result_s7 = result.rolling(7).sum()

# Inzidenz berechnen:

for i in result_s7.columns:
    result_s7[i] = result_s7[i]*100000/ float(einwohner.loc[i])
    

if DEBUG:
    overview=['Deutschland', 'Bayern', 'SK München', 'SK Duisburg']
    for i in overview:
        print('Inzidenz ',i, '\t\t', f'{result_s7.iloc[-1][i]:.1f}')

In [ ]:
fig = plt.figure(figsize=[15.,25.])

#First figure:

cx=fig.add_subplot(4,1,1)

cx.grid()
a=cx.set_title('Neuinfektionen',fontsize=16)
b=cx.set_xlabel('Datum', fontsize=12)
c=cx.set_ylabel('Inzidenz', fontsize=12)


cx.plot(result_s7['Deutschland'], label='Deutschland', linewidth=3)

overview=['Bayern', 'SK München', 'SK Duisburg']
for i in overview:
    cx.plot(result_s7[i], label=i)


e=cx.set_xlim([datetime(2020,8,1),datetime(2021,12,31)])
f=cx.set_yscale('log')

g=cx.set_ylim([1,1000])
#ax.set_yticks([1, 10, 100, 1000])
cx.get_yaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
h=cx.legend(loc='best', fontsize=12)

In [ ]:
# County Bezeichnungen zum gpd File dazumergen

tmpkreise = kreise.copy()
tmpkreise.RS = tmpkreise.RS.astype(int)
tmpewz = ewz.loc[:,['RS','county']]
#tmpkreise = tmpkreise.merge(tmpewz, on='RS')
tmpkreise = pd.merge(tmpkreise,tmpewz, on='RS', how='right')

# Was soll dargestellt werden:

a=pd.DataFrame(result_s7.iloc[-1])
b=pd.DataFrame((result_s7.iloc[-1] - result_s7.iloc[-8])/result_s7.iloc[-8]*100)


a.rename(columns={ a.columns[0]: "INZ" }, inplace = True)
a.reset_index(inplace=True)
a.rename(columns={'index': 'county'},inplace=True)

b.rename(columns={ b.columns[0]: "DINZ" }, inplace = True)
b.reset_index(inplace=True)
b.rename(columns={'index': 'county'},inplace=True)


#Rechenergebnis zum Plotfile hinzumergen

tmpkreise=pd.merge(tmpkreise, a, on='county', how='left')
tmpkreise=pd.merge(tmpkreise, b, on='county', how='left')

#Berlin Korrektur
berlin_ges=kreise[kreise['RS']=='11000']
tmpkreise=tmpkreise.append(berlin_ges)
tmpkreise.loc[tmpkreise['RS']=='11000','county']='Berlin'
tmpkreise.loc[tmpkreise['RS']=='11000','INZ']=float(a.loc[a.county=='Berlin'].INZ)
tmpkreise.loc[tmpkreise['RS']=='11000','DINZ']=float(b.loc[a.county=='Berlin'].DINZ)

In [ ]:
#Mainplot mit Bundeslandgrenzen

cm=1/2.54
fig = plt.figure(figsize=(60*cm, 38*cm))
ax=fig.add_subplot(1, 2, 1)

#ax = tmpkreise.plot(figsize=(15,15), edgecolor='grey', column='INZ', cmap='tab10', scheme='MaximumBreaks', legend=True)
#ax = tmpkreise.plot(figsize=(15,15), edgecolor='grey', column='INZ', cmap='tab10', scheme='Quantiles', legend=True)


bins1 =[5, 20, 35, 50, 100, 200, 300, 1000, 2500]
px1 = tmpkreise.plot(ax=ax, edgecolor='grey', column='INZ', cmap='GnBu', 
                    scheme='UserDefined', classification_kwds={'bins': bins1},
                    legend=True)

px2=bland.plot(ax=ax, edgecolor='white', color='none')


txt="Inzidenzkarte Deutschland "
end_str=ende.strftime('%d.%m.%Y')
txt += ("am " + end_str + ':\n')
cx=ax.text(5.5,55.1, txt,fontsize=16)

txt="(7 Tage pro 100.000 Einwohner)"
dx=ax.text(5.5,55.05,txt)

copyright2 = "Datenquellenvermerk: Robert Koch-Institut (RKI), dl-de/by-2-0"
ex=ax.text(5.5,47,copyright2)





bx=fig.add_subplot(1, 2, 2)
#dbins =[-100, -50, 0, 50, 100, 200]
dbins =[0, 50, 100, 200]
px3 = tmpkreise.plot(ax=bx, edgecolor='grey', column='DINZ', cmap='tab10', 
                    scheme='UserDefined', classification_kwds={'bins': dbins},
                    legend=True)

px4=bland.plot(ax=bx, edgecolor='white', color='none')

txt="Inzidenzänderung zur Vorwoche in Prozent "
end_str=ende.strftime('%d.%m.%Y')
txt += (", " + end_str + ':\n')
cx=bx.text(5.5,55.1, txt,fontsize=16)


copyright2 = "Datenquellenvermerk: Robert Koch-Institut (RKI), dl-de/by-2-0"
ex=bx.text(5.5,47,copyright2)


pre_str=ende.strftime('%Y%m%d')
diagram_file=pre_str+"_Inzidenzkarte_Deutschland"
plt.savefig(diagram_file, dpi=300)

In [ ]:
#One Diagram

fig = plt.figure(figsize=(38*CM, 60*CM))

bx=fig.add_subplot(1, 1, 1)
#dbins =[-100, -50, 0, 50, 100, 200]
dbins =[0, 50, 100, 200]
px3 = tmpkreise.plot(ax=bx, edgecolor='grey', column='DINZ', cmap='tab10', 
                    scheme='UserDefined', classification_kwds={'bins': dbins},
                    legend=True)

px4=bland.plot(ax=bx, edgecolor='white', color='none')

txt="Inzidenzänderung zur Vorwoche in Prozent "
end_str=ende.strftime('%d.%m.%Y')
txt += (", " + end_str + ':\n')
cx=bx.text(5.5,55.1, txt,fontsize=16)


copyright2 = "Datenquellenvermerk: Robert Koch-Institut (RKI), dl-de/by-2-0"
ex=bx.text(5.5,47,copyright2)


pre_str=ende.strftime('%Y%m%d')
diagram_file=pre_str+"_Delta-Inzidenzkarte_Deutschland"
plt.savefig(diagram_file, dpi=300)

In [ ]:
# Top Änderungen

deltas=tmpkreise[['county','DINZ']].set_index('county').DINZ
deltas.drop_duplicates(inplace=True)
deltas.drop('Berlin',inplace=True)

#Statistics
m=deltas.mean()
neg=len(deltas[deltas<0])
pos=len(deltas[deltas>=0])

Summary = 'Landkreise fallend: ' + str(neg) + '\nLandkreise steigend: '+ str(pos) + '\nMittlere Änderung ' + '{:.2f}'.format(m) + '%'

#Histogramm

#hp=deltas.hist(bins=41)
hp=deltas.hist()

y=hp.set_ylabel('Antzahl Land-/Stadtkreise')
y=hp.set_xlabel('Prozentuale Änderung der 7 Tage Inzidenz/100T EW ggü. Vorwoche')

y=hp.set_title('Anzahl Landkreise über Inzidenzändeung in %')
txt='Stand '+ ende.strftime('%d.%m.%Y')
y=hp.text(25,12,txt)
y=hp.text(-60,-40,copyright2)

y=hp.text(10,110, Summary)

histplotfile=ende.strftime('%y%m%d')+' Histogramm'
plt.savefig(histplotfile)

In [ ]:
### Debug ###

In [ ]:
ewz[ewz['county'].str.contains('Würz')]